In [ ]:
import numpy as np
import random
import pulp as plp
import itertools
from itertools import product
import random
from decimal import Decimal, ROUND_HALF_UP

def round_to_two_decimal_places(value):
    #Rounds a value to two decimal places with ROUND_HALF_UP.
    return Decimal(value).quantize(Decimal("0.01"), rounding=ROUND_HALF_UP)

In [ ]:
class pnet:
    """
    Represents a Time Petri Net.

    Attributes:
    - transitions: List of transitions in the Petri net.
    - places: List of places in the Petri net.
    - arcs: List of arcs in the Petri net.

    To build a petri net, the input has to be in the following format:
      atmg = pnet(arcs, intervals)

    - where the arcs parameter is a list of couples of strings, representing the set F of the TPN (i.e. couples of place-transition or transition-place)
      Places have to start with the letter p, transitions with the letter t followed by a number that indicates the order of the list of intervals, except for the number 0.    
      e.g.: arcs = [("p0", "t1"), ("t1", "p1"), ("t1", "p2"), ("p1", "t2"), ("p2", "t3"), ("t2","p3"), ("t3","p4") ,("p3", "t4") , ("p4", "t4"), ("t4", "p5")],
       
    - where intervals is a list of 2d lists that represent the intervals for each transition, in the order defined by the name of the transitions as defined before 
      e.g. intervals = [[0, 7], [0, 5], [0, 3], [0,1], [0,3], [1,3 ], [1,5] , [0,55], [0,66]] 
    
    Therefore, given pnet(arcs, intervals), the transition t1 is gonna have interval [0,7] (first in the list), t2 has [0,5] and so on.
   """
    transitions = []
    places = []
    arcs = []
    def __init__(self, arcs, intervals):

        self.transitions = []
        self.places = set()
        self.arcs = arcs

        transitions = set()
        for arc in self.arcs:
            if arc[0].startswith('p'):  # If it's a place, add it to the places set
                self.places.add(arc[0])
            elif arc[0].startswith('t'):  # If it's a transition, add it to the transitions set
                transitions.add(arc[0])

            if arc[1].startswith('p'):  # If it's a place, add it to the places set
                self.places.add(arc[1])
            elif arc[1].startswith('t'):  # If it's a transition, add it to the transitions set
                transitions.add(arc[1])

        # Create a list of transitions ordered by transition number (extracted from the transition name)
        ordered_transitions = sorted(transitions, key=lambda t: int(t[1:]))

        # Create the transitions + intervals vector
        self.transitions = []
        for t in ordered_transitions:
            # Since transitions are ordered from t1 to t30, we can get the corresponding interval from the intervals vector
            index = int(t[1:]) - 1  # The index for intervals is 1-based, so subtract 1 to get 0-based index
            self.transitions.append((t, intervals[index]))

        self.order_transitions()

    def __str__(self):
        return f"Places: {self.places} \nTransitions: {self.transitions} \nArcs: {self.arcs}"

    def get_interval(self, idx_transition):
        #returns the timestamp interval of a transition
        return self.transitions[idx_transition][1]

    def get_parents(self, idx_transition):
        #returns the direct parents of a transition in the petri net

        parents = []
        places = []
        for arc in self.arcs:
            if arc[1] == self.transitions[idx_transition-1][0]:
                if arc[0] not in places:
                    places.append(arc[0])

        for arc in self.arcs:
            if arc[1] in places:
                if arc[0] not in parents:
                    parents.append(arc[0])
        return parents

    def get_children(self, idx_transition):
        #returns the direct successors of a transition in the petri net

        children = []
        places = []
        for arc in self.arcs:
            if arc[0] == self.transitions[idx_transition-1][0]:
                if arc[1] not in places:
                    places.append(arc[1])

        for arc in self.arcs:
            if arc[0] in places:
                if arc[1] not in children:
                    children.append(arc[1])
        return children

    def idx(self, transition):
        #returns the transition index
        return int(transition[1:])

    def dimension(self):
        #returns the dimension (#transitions) of the TPN
        return len(self.transitions)

    def retrieve_mapping(self, temp):
        #returns a mapping of the old and new names of the transitions of the petri net

        mapping = dict()

        partial_transitions = [t[0] for t in self.transitions]
        #print(partial_transitions)

        for i,transition in enumerate(self.transitions):
            after = transition[0]
            before = temp[partial_transitions.index(after)]
            #print(before, after)
            mapping[before] = after
        return mapping

    def modify_arcs(self, mapping):
        #modifies the arcs with the renamed transitions of the sorted petri net

        for i, arc in enumerate(self.arcs):
            initial = arc[0]
            final = arc[1]

            #if initial is a string that starts with "t"
            if initial.startswith("t"):
                initial = mapping[initial]

            #if final is a string that starts with "t"
            if final.startswith("t"):
                final = mapping[final]

            arc = (initial, final)
            self.arcs[i] = arc

    def order_transitions(self):
        #orders and renames the transitions of the generated petri net

        ordered_transitions = []
        visited = set()

        def dfs(transition):
            visited.add(transition)
            for child in self.get_children(self.idx(transition)):
                if child not in visited:
                    dfs(child)
            ordered_transitions.append(transition)

        for transition in self.transitions:
            if transition[0] not in visited:
                dfs(transition[0])

        ordered_transitions.reverse()
        mapping = self.retrieve_mapping(ordered_transitions)


        #print(mapping)
        temp = [("t"+str(i+1), self.get_interval(self.idx(transition)-1)) for i,transition in enumerate(ordered_transitions)]

        self.transitions = temp
        self.modify_arcs(mapping)

        return self.transitions

    def find_idx_transition(self, transition):
        #returns the index in the transitions list of the specified transition name
        
        #print(transition)
        only_transitions_names = [t[0] for t in self.transitions]
        return only_transitions_names.index(transition)

    def generate_random_log(self, cardinality):
        #Generates a log with n = cardinality random (a random value taken from every interval) traces accepted by the model
        log = []

        for _ in range(cardinality):
            ex = []
            for i in range(len(self.transitions)):
                if len(self.get_parents(i + 1)) == 0:
                    # Generate a random value within the interval and round it to two decimal places
                    value = random.uniform(self.get_interval(i)[0], self.get_interval(i)[1])
                    rounded_value = round_to_two_decimal_places(value)
                    ex.append(float(rounded_value))
                else:
                    # Get the maximum value from the parent transitions and add to the random value
                    list_of_parents_idxs = [self.find_idx_transition(parent) for parent in self.get_parents(i + 1)]
                    parent_value = max([ex[idx] for idx in list_of_parents_idxs])

                    # Generate the new value and ensure it's rounded to two decimal places
                    new_value = random.uniform(self.get_interval(i)[0], self.get_interval(i)[1]) + parent_value
                    rounded_value = round_to_two_decimal_places(new_value)
                    ex.append(float(rounded_value))
            log.append(ex)

        return log

    def max_point(self):
        #returns the maximal point \sigma_M of the model 
        max_point = []

        for i in range(len(self.transitions)):
            if len(self.get_parents(i+1)) == 0:
                max_point.append(self.get_interval(i)[1])
            else:
                list_of_parents_idxs = [self.find_idx_transition(parent) for parent in self.get_parents(i+1)]
                parent_value = max([max_point[i] for i in list_of_parents_idxs])
                max_point.append( self.get_interval(i)[1] + parent_value)
        return max_point

    def min_point(self):
        #returns the minimal point \sigma_m of the model 

        min_point = []

        for i in range(len(self.transitions)):
            if len(self.get_parents(i+1)) == 0:
                min_point.append(self.get_interval(i)[0])
            else:
                list_of_parents_idxs = [self.find_idx_transition(parent) for parent in self.get_parents(i+1)]
                parent_value = max([min_point[i] for i in list_of_parents_idxs])
                min_point.append(self.get_interval(i)[0] + parent_value)
        return min_point

    pass

def transform_log(L, model):
    #Transforms the log into the equivalent using flow functions (for delay only distance)
    for i, trace in enumerate(L):
        # Convert each tuple to a list
        if len(model.transitions)==1:
            trace_list = list([trace])
        else:
            trace_list = list(trace)

        old_trace = trace_list.copy()


        for j in range(len(trace_list)):
            #print(trace_list[j])
            if len(model.get_parents(j+1)) != 0:
                list_of_parents_idxs = [model.find_idx_transition(parent) for parent in model.get_parents(j+1)]
                parent_value = max([old_trace[i] for i in list_of_parents_idxs])
                trace_list[j] = trace_list[j] - parent_value

            # Convert the list back to a tuple
            L[i] = tuple(trace_list)

    return L

def search_space_type1(counter, dimension, brought, model, gamma, search_space,step, distance_type):
    #build the search space (set of "all" points accepted by the model) brute force, for a ATMG as described in the experiments section of the paper
    #as the space is continuous, the parameter "step" determines the number of splits of the space of each dimension into equidistant points

  if counter-1 == dimension:
      # Base case: all loops have been executed, do something with the values
      search_space.append(gamma.copy())
      #print(gamma, search_space)
      return search_space
  else:
      lower = model.get_interval(counter-1)[0] + brought
      upper = model.get_interval(counter-1)[1] + brought
      for i in  np.linspace(lower, upper, step):
          gamma.append(i)  # Modify the values for the current loop

          if distance_type == 0:
              if(counter in range(2,dimension)):
                if counter == dimension-1:
                  give = np.max(gamma[1:dimension-1])
                else:
                  give = gamma[0]
                #print(counter, give)
              elif(counter == 1):
                give = i
              elif(counter == dimension):
                give = np.max(gamma[1:dimension-1])
                #print(gamma, give, counter)
              else:
                give = np.inf
                print("error")
              search_space_type1(counter + 1, dimension, give, model , gamma, search_space, step, distance_type)  #Recursively call the function for the next loop
          else:
              search_space_type1(counter + 1, dimension, 0, model , gamma, search_space, step, distance_type)
          gamma.pop()  # Remove the current loop's value

  if counter == 1:
      print('done')
      return search_space

def search_space_delay(model, step):
  #builds the search space when using the delay only distance
  #(discretized, still with the parameter step determining the number of splits of the space of each dimension into equidistant points)

  # Create a list of linspaces for each interval
  linspaces = [np.linspace(t[1][0], t[1][1], step) for t in model.transitions]

  # Generate all combinations of points using itertools.product
  combinations = set(product(*linspaces))

  return combinations

def find_aa(model, L, step, distance_type):
    #finds all possible anti-alignments considering the discretized search space, and comparing every point brute force
    
    dimension = len(model.transitions)
    #print('dimension:', dimension)

    if distance_type == 0:
        search_space = search_space_type1(1, dimension, 0, model, [], [], step, distance_type)
    else:
        search_space = search_space_delay(model, step)

    print(model)
    #print('Search space:', search_space)


    # Initialize variables
    max_distance = float('-inf')
    max_points = []

    # Convert the list L to a numpy array for efficient vector operations
    L_array = np.array(L)

    # Iterate over each point in the search space
    for gamma in search_space:
        # Convert gamma to a numpy array for efficient vector operations
        gamma_array = np.array(gamma)

        # Calculate the minimum distance for the current gamma
        distances = np.linalg.norm(L_array - gamma_array, ord=1, axis=1)  # Efficient L1 norm computation
        min_distance = np.min(distances)  # Minimum distance to any point in L

        # Update max_distance and max_points based on the minimum distance
        if min_distance > max_distance:
            max_distance = min_distance
            max_points = [gamma]
        elif min_distance == max_distance:
            max_points.append(gamma)

    return max_distance, max_points

def brute_force_solver(model, log, step, distance_type):
    #the log is to be transformed or not depending on whether the LP solver has already been used for the same log
    #in such case, the log is already transformed
    #if the BF solver is the only one used, this line has to be uncommented 
    """
    if distance_type == 1:
        log = transform_log(log, model)
    """
    max_distance, max_points = find_aa(model, log, step, distance_type)
    #print('Max Distance:', max_distance)
    #print('Optimal Point:', max_points[0])

    return max_distance, max_points


def gen_pnet_type0(dimension,random_intervals=False):
    #returns a petri net with all transitions in parallel and not linked to each other

    for i in range(dimension):
        if i == 0:
            arcs = [("p00", "t1"), ("t1", "p01")]
            if random_intervals:
                lower_bound =  round(round(random.uniform(0, 10),3), 2)
                upper_bound =  lower_bound + round(round(random.uniform(0, 10),3), 2)
                intervals= [(lower_bound, upper_bound)]
            else:
                intervals = [(0, 1)]
        else:
            arcs.append(("p"+str(i)+ "0", "t"+str(i+1)))
            arcs.append(("t"+str(i+1), "p"+str(i)+ "1"))
            if random_intervals:
                lower_bound =  round(round(random.uniform(0, 10),3), 2)
                upper_bound =  lower_bound + round(round(random.uniform(0, 10),3), 2)
                intervals.append((lower_bound, upper_bound))
            else:
                intervals.append((0, 1))


    model = pnet(arcs, intervals)
    print(model)
    return model

import json
import ast

def retrieve_example(cardinality, dimension):
    #retrieves the log and model from the JSON file given the cardinality and dimension of the problem
    #it retrieves it only for distance type 0 (stamp only), to retrieve the "original" log
        
    filepath = "/content/drive/MyDrive/BF_PAR_lp_solver_logs.json"

    # Load JSON data from a file
    with open(filepath, "r") as f:
        json_data = json.load(f)

    key_pattern = f"{cardinality}-{dimension}-0"

    # Find the entry with the corresponding key
    for key, entry in json_data.items():
        if entry.get("Key") == key_pattern:
            # Get the Model and Log from the "Log Data"
            model = ast.literal_eval(entry.get("Log Data", {}).get("Model"))
            log = entry.get("Log Data", {}).get("Log")
            return model, log

    # If the key pattern is not found, return a message
    return {"error": "No data found for the given key pattern"}

def LPSolver(model, log, distance_type=0):
    #Linear Programming solver with constraints and variables as described in the paper

    # Define the dimensionality of the problem
    n = model.dimension() #dimension of the space
    m = len(log) #number of points in L

    if distance_type ==1 : #if delay-only distance is used, the log is transformed in equivalent flow functions traces
      log = transform_log(log,model)

    #print(model.transitions, log[0])

    # Create the LP problem

    prob = plp.LpProblem("Maximize minimal manhattan distance", plp.LpMaximize)

    # Define the decision variables

    x = plp.LpVariable.dicts("x", range(n), lowBound=0, cat = 'Continuous')

    #Define the variable to maximize
    z = plp.LpVariable("z", lowBound=0, cat = 'Continuous')

    #print(x, z)

    
    M = np.linalg.norm(np.array(model.max_point()) - np.array(model.min_point()), ord=1)
    print('M', M)

    # Define the constraints for x, i.e. the search space constraints
    if distance_type == 0:
        max_var = plp.LpVariable.dicts("max_var", range(n), cat = 'Continuous')
        for i in range(n):
            #print(i)
            if len(model.get_parents(i+1)) == 0:
                prob += x[i] >= model.get_interval(i)[0]
                prob += x[i] <= model.get_interval(i)[1]
            else:
                binary = plp.LpVariable.dicts("binary_"+str(i), range(len(model.get_parents(i+1))), cat="Binary")
                for j, parent in enumerate(model.get_parents(i+1)):
                    prob+= max_var[i] >= x[model.idx(parent)-1]
                    prob+= max_var[i] <= x[model.idx(parent)-1] + (1-binary[j])*M
                    prob += x[i] >= model.get_interval(i)[0] + x[model.idx(parent)-1]
                prob += plp.lpSum([binary[j] for j in range(len(model.get_parents(i+1)))]) == 1
                prob += x[i] <= model.get_interval(i)[1] + max_var[i]
    else:
        for i in range(n):
            prob += x[i] >= model.get_interval(i)[0]
            prob += x[i] <= model.get_interval(i)[1]


    # Define the constraints for the absolute value of the difference between x and each point in L
    diff_plus = plp.LpVariable.dicts("diff_plus", (range(m), range(n)), lowBound=0, cat = 'Continuous')
    diff_minus = plp.LpVariable.dicts("diff_minus", (range(m), range(n)), lowBound=0, cat = 'Continuous')

    b = plp.LpVariable.dicts("b", (range(m), range(n)), cat = 'Binary')


    for j in range(m): #for every sigma in L
        for i in range(n): #for every dimension
            if model.dimension() == 1: #if the model is 1-dimensional
                prob += diff_plus[j][i] - diff_minus[j][i] == log[j] - x[i]
            else:
                prob += diff_plus[j][i] - diff_minus[j][i] == log[j][i] - x[i]

            prob += diff_plus[j][i] <= M*b[j][i]
            prob += diff_minus[j][i] <= M*(1-b[j][i])

        prob += z <= plp.lpSum([diff_plus[j][i] + diff_minus[j][i] for i in range(n)])


    # Define the objective function
    prob += z

    #print(prob)

    # Solve the problem
    prob.solve()

    # Print the results
    print("Status:", plp.LpStatus[prob.status])
    #print("Max Distance:", plp.value(prob.objective))

    optimal_point = [plp.value(x[i]) for i in range(n)]
    #print('Optimal Point:', optimal_point)

    return plp.value(prob.objective), optimal_point, prob.numVariables(), prob.numConstraints()

def gen_pnet_type1(dimension, random_intervals):
    #generates a random ATMG with the given dimension, with the structure and the extrema of the intervals as explained in the paper (experiments section)

    for i in range(dimension):
        if i == 0:
            arcs = [("p00", "t1")]
            if random_intervals:
                lower_bound =  round(round(random.uniform(0, 5),3), 2)
                upper_bound =  lower_bound + round(round(random.uniform(0, 5),3), 2)
                intervals= [(lower_bound, upper_bound)]
            else:
                intervals = [(0, 1)]
        elif i == dimension-1:
            arcs.append(("t"+str(i+1), "p"+str(i)+ "1"))
            if random_intervals:
                lower_bound =  round(round(random.uniform(0, 5),3), 2)
                upper_bound =  lower_bound + round(round(random.uniform(0, 5),3), 2)
                intervals.append((lower_bound, upper_bound))
            else:
                intervals.append((0, 1))
        else:
            arcs.append(("t1", "p0"+str(i)))
            arcs.append(("p0"+str(i), "t"+str(i+1)))
            arcs.append(("t"+str(i+1), "p"+str(i)+ "1"))
            arcs.append(("p"+str(i)+ "1", "t"+str(dimension)))
            if random_intervals:
                lower_bound =  round(round(random.uniform(0, 5),3), 2)
                upper_bound =  lower_bound + round(round(random.uniform(0, 5),3), 2)
                intervals.append((lower_bound, upper_bound))
            else:
                intervals.append((0, 1))

    print(arcs, intervals)
    model = pnet(arcs, intervals)
    #print(model)
    return model

def generate_random_example_t1(dimension,cardinality,random_model):
        #generates a random example of problem given the parameters of dimension and cardinality, 
        #i.e. an ATMG with #trns = dimension (and structure as described in the experiment section of the paper) 
        #and a random log accepted by the model with #traces = cardinality
        # if random_model = False, then the model will have all intervals as [0,1]
        model = gen_pnet_type1(dimension,random_model)

        log = model.generate_random_log(cardinality)
        return log, model



In [ ]:
import time
import json
import pandas as pd
from itertools import product
import os

# Parameters to generate examples
cardinalities = [10]
dimensions = [12]
distance_types = [0,1]
#step = 10

# Generate list of pairs of cardinalities and dimensions
pairs = product(cardinalities, dimensions)

# File paths for storing results
json_filename = "/content/drive/MyDrive/BF_PAR_solver_logs.json"
csv_filename = "/content/drive/MyDrive/BF_PAR_solver_results.csv"
json_filename2 = "/content/drive/MyDrive/BF_PAR_solver_logs2.json"

def save_log_to_json(key, log_data, file_path):
    #Saves log and model to a JSON file with a unique identifier, along with the key
    
    # Convert the data to a serializable format
    log_data_serializable = {str(k): v for k, v in log_data.items()}

    # Generate a unique identifier for the log
    log_id = hash(json.dumps(log_data_serializable, sort_keys=True))

    # Structure for storing the log with the key
    log_entry = {
        "Key": key,
        "Log Data": log_data_serializable
    }

    # Create or update the JSON file
    if not os.path.exists(file_path):
        with open(file_path, 'w') as f:
            json.dump({}, f, indent=4)

    # Load existing data
    with open(file_path, 'r') as f:
        existing_data = json.load(f)

    # Add the new log with the unique identifier
    existing_data[str(log_id)] = log_entry

    # Write back the updated data
    with open(file_path, 'w') as f:
        json.dump(existing_data, f, indent=4)

    return log_id  # Return the unique identifier to reference this log

# Function to append to a CSV file
def append_results_to_csv(results_dict, file_path):
    #Appends data (result) to a CSV file. 
    df = pd.DataFrame([results_dict])
    file_exists = os.path.exists(file_path)

    # If the file does not exist or is empty, write headers
    if not file_exists or os.path.getsize(file_path) == 0:
        df.to_csv(file_path, mode='w', index=False, header=True, sep=';')
    else:
        df.to_csv(file_path, mode='a', index=False, header=False, sep=';')  # Append without headers


# Dictionary to store results
lp_results = {}

# LP-based experiments loop
for cardinality, dimension in pairs:

    # Generate\retrieve example 
    #model, log, = retrieve_example(cardinality, dimension)
    log, model = generate_random_example_t1(dimension, cardinality, random_model = True)

    #print(model)


    for distance_type in distance_types:
        key = f"{cardinality}-{dimension}-{distance_type}"
        print(f"Cardinality: {cardinality}, Dimension: {dimension}, Distance Type: {distance_type}")

        # Save the log data to the JSON file and get the unique ID
        log_id = save_log_to_json(key, {"Model": str(model), "Log": log}, json_filename2)
        #REMOVE LOG AND MODEL FROM JSON IF SOLUTION NOT ENDED

        start_time = time.time()
        max_dist_LP, opt_point_LP, numVars, numConstr = LPSolver(model, log, distance_type)
        elapsed_time_LP = time.time() - start_time

        start_time_BF = time.time()
        max_dist_BF, opt_point_BF = brute_force_solver(model, log, 5, distance_type)
        elapsed_time_BF = time.time() - start_time_BF

        # Save the log data to the JSON file and get the unique ID
        log_id = save_log_to_json(key, {"Model": str(model), "Log": log}, json_filename)
        #REMOVE LOG AND MODEL FROM JSON IF SOLUTION NOT ENDED


        # Store the results
        result_data = {
            'Key': key,
            'Dimension': dimension,
            'Cardinality': cardinality,
            'Distance Type': distance_type,
            #'Model': str(model),  # Ensure this is serializable
            #'Log': str(L),  # Ensure this is serializable
            'Elapsed Time (LP)': elapsed_time_LP,
            'Elapsed Time (BF)': elapsed_time_BF,
            #'Number of Variables (LP)': numVars,
            #'Number of Constraints (LP)': numConstr,
            'Solution (LP)': str(opt_point_LP),  # Ensure this is serializable
            'Solution (BF)': str(opt_point_BF),  # Ensure this is serializable
            'Max Distance (LP)': max_dist_LP,
            'Max Distance (BF)': max_dist_BF,
        }

        append_results_to_csv(result_data, csv_filename)


        # Display the results
        print(f"Elapsed Time (LP): {elapsed_time_LP:.6f} seconds")
        print(f"Elapsed Time (BF): {elapsed_time_BF:.6f} seconds")
        #print(f"Number of Variables (LP): {numVars}")
        #print(f"Number of Constraints (LP): {numConstr}")
        print(f"Max Distance (LP): {max_dist_LP}")
        print(f"Max Distance (BF): {max_dist_BF}")

        print()

        # Save the results to both JSON and CSV files
        #append_results_to_json(lp_results, json_filename)

print(f"LP-based results successfully appended {csv_filename}.")

[('p00', 't1'), ('t1', 'p01'), ('p01', 't2'), ('t2', 'p11'), ('p11', 't12'), ('t1', 'p02'), ('p02', 't3'), ('t3', 'p21'), ('p21', 't12'), ('t1', 'p03'), ('p03', 't4'), ('t4', 'p31'), ('p31', 't12'), ('t1', 'p04'), ('p04', 't5'), ('t5', 'p41'), ('p41', 't12'), ('t1', 'p05'), ('p05', 't6'), ('t6', 'p51'), ('p51', 't12'), ('t1', 'p06'), ('p06', 't7'), ('t7', 'p61'), ('p61', 't12'), ('t1', 'p07'), ('p07', 't8'), ('t8', 'p71'), ('p71', 't12'), ('t1', 'p08'), ('p08', 't9'), ('t9', 'p81'), ('p81', 't12'), ('t1', 'p09'), ('p09', 't10'), ('t10', 'p91'), ('p91', 't12'), ('t1', 'p010'), ('p010', 't11'), ('t11', 'p101'), ('p101', 't12'), ('t12', 'p111')] [(0.12, 5.08), (3.15, 6.77), (1.14, 4.36), (2.44, 6.970000000000001), (0.01, 3.5199999999999996), (0.37, 4.5200000000000005), (0.97, 4.03), (3.45, 5.07), (0.47, 4.32), (4.91, 7.6), (0.59, 3.5), (2.15, 3.27)]
Cardinality: 10, Dimension: 12, Distance Type: 0
[('t1', (0.12, 5.08)), ('t2', (0.59, 3.5)), ('t3', (4.91, 7.6)), ('t4', (0.47, 4.32)), ('t5'